In [1]:
# v3.classification
# 3/12/2018

import argparse
import os
import glob
import numpy as np
import cv2
import torch
import sklearn.feature_extraction.image
from torchvision.models import DenseNet
import tables
#-----helper function to split data into batches


# ----- parse command line arguments
# parser = argparse.ArgumentParser(description='Make output for entire image using Unet')
# parser.add_argument('input_pattern',
#                     help="input filename pattern. try: *.png, or tsv file containing list of files to analyze",
#                     nargs="*")

# parser.add_argument('-p', '--patchsize', help="patchsize, default 256", default=256, type=int)
# parser.add_argument('-s', '--batchsize', help="batchsize for controlling GPU memory usage, default 10", default=10, type=int)
# parser.add_argument('-o', '--outdir', help="outputdir, default ./output/", default="./output/", type=str)
# parser.add_argument('-r', '--resize', help="resize factor 1=1x, 2=2x, .5 = .5x", default=1, type=float)
# parser.add_argument('-m', '--model', help="model", default="best_model.pth", type=str)
# parser.add_argument('-i', '--gpuid', help="id of gpu to use", default=0, type=int)
# parser.add_argument('-f', '--force', help="force regeneration of output even if it exists", default=False,
#                     action="store_true")
# parser.add_argument('-b', '--basepath',
#                     help="base path to add to file names, helps when producing data using tsv file as input",
#                     default="", type=str)

# args = parser.parse_args()
# #args

# if not (args.input_pattern):
#     parser.error('No images selected with input pattern')

In [3]:
def divide_batch(l,hc, n): 
    for i in range(0, l.shape[0], n):
        x = l[i:i + n,:]
        y = hc[i:i + n,:]
        yield x , y

In [7]:
OUTPUT_DIR = 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\OrganSiteClassifying\\FusionNew\\Test1Out\\'
resize = 1
# batch_size = args.batchsize
batch_size = 100
patch_size = 376
stride_size = 376//2
base_path = 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\OrganSiteClassifying\\Test1\\**\\'
input_pattern = '*.png'
mdlPath = 'CUPBinlinFusion_densenet_best_model.pth'

In [8]:
import torch.nn as nn
import torch.nn.functional as F

# increasing few layers in our model
class DenseNetFusion(nn.Module):
    def __init__(self):
        super(DenseNetFusion, self).__init__()
        self.image_dense_layer_1 = nn.Linear(300 , 100)
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm1d(100)
        self.dropout = nn.Dropout2d(0.5)
        self.image_dense_layer_2 = nn.Linear(100, 10)

        
        
        self.BilinearPool = nn.Bilinear(10, 10, 10)
        self.final = nn.Linear(10 , 4)
        self.tabular_dense_layer_1 = nn.Linear(10, 10)
#         self.tabular_dense_layer_2 = nn.Linear(8, 5)        
        
    
        
        
    def forward(self, image_inputs , tabular_data_inputs):
#       DL data
        x_img = image_inputs
        x_img = self.image_dense_layer_1(x_img)
        x_img = self.relu(x_img)
        x_img = self.batchnorm1(x_img)
        x_img = self.dropout(x_img)
        x_img = self.image_dense_layer_2(x_img)
        x_img = self.relu(x_img)

        
#       Tabular data
        tab = self.tabular_dense_layer_1(torch.tensor(tabular_data_inputs, device=device).float())
        tab = self.relu(tab)
        x = self.BilinearPool(x_img,tab)
#         x = torch.cat((x, tab), dim=1)

        return self.final(x)

In [9]:
# ----- load network
device = torch.device(0 if 0!=-2 and torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(mdlPath, map_location=lambda storage, loc: storage) #load checkpoint to CPU and then put to device https://discuss.pytorch.org/t/saving-and-loading-torch-models-on-2-machines-with-different-number-of-gpu-devices/6666
model = DenseNetFusion()
model = model.to(device)
# model = DenseNet(growth_rate=checkpoint["growth_rate"], block_config=checkpoint["block_config"],
#                  num_init_features=checkpoint["num_init_features"], bn_size=checkpoint["bn_size"],
#                  drop_rate=checkpoint["drop_rate"], num_classes=checkpoint["num_classes"]).to(device)
# model.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# model.cuda(0)


print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

# ----- get file list

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

files = []
# basepath = args.basepath  #
# basepath = basepath + os.sep if len(
#     basepath) > 0 else ""  # if the user supplied a different basepath, make sure it ends with an os.sep

# if len(args.input_pattern) > 1:  # bash has sent us a list of files
#     files = args.input_pattern
# elif args.input_pattern[0].endswith("tsv"):  # user sent us an input file
#     # load first column here and store into files
#     print('loading tsv file...')
#     with open(args.input_pattern[0], 'r') as f: #added encoding="ISO-8859-1" to fix UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte
#         for line in f:
#             print(line)
#             if line[0] == "#":
#                 continue
#                 print('Continue')
#             files.append(basepath + line.strip().split("\t")[0])
# else:  # user sent us a wildcard, need to use glob to find files
#     files = glob.glob(args.basepath + args.input_pattern[0])
files = glob.glob(base_path + input_pattern)

total params: 	32474


In [10]:
testTable = tables.open_file('CUPBinlinFusion_Test.pytable')

In [11]:
# base_path + input_pattern
# files = [ 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_12033.png','E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13569.png',
#  'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13825.png']
os.path.basename(testTable.root.filenames[0].strip())[0:-4]

b'79M_100353_12289_43'

In [32]:
model.load_state_dict(checkpoint["model_dict"])
model.eval()

DenseNetFusion(
  (image_dense_layer_1): Linear(in_features=300, out_features=100, bias=True)
  (relu): ReLU()
  (batchnorm1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout2d(p=0.5, inplace=False)
  (image_dense_layer_2): Linear(in_features=100, out_features=10, bias=True)
  (BilinearPool): Bilinear(in1_features=10, in2_features=10, out_features=10, bias=True)
  (final): Linear(in_features=10, out_features=4, bias=True)
  (tabular_dense_layer_1): Linear(in_features=10, out_features=10, bias=True)
)

In [33]:
testTable.root.DLfeats.shape

(14502, 300)

In [34]:
testTable.root

/ (RootGroup) ''
  children := ['DLfeats' (EArray), 'HCFeats' (EArray), 'classsizes' (CArray), 'filenames' (EArray), 'imgs' (EArray), 'labels' (EArray)]

In [35]:
# +
# ------ work on files
#print(files)
saveScore = []
from tqdm import tqdm_notebook
classPred= open(OUTPUT_DIR+"\classPred.txt","w+")
classPredScore= open(OUTPUT_DIR+"\classPredScore.txt","w+")
classScore = open(OUTPUT_DIR+"\classScore.txt","w+")
classPredDetailed = open(OUTPUT_DIR+"\classPredDetailed.txt","w+")
for i in tqdm_notebook(range(testTable.root.imgs.shape[0])):
    
    fname = testTable.root.filenames[i].strip()
    
    newfname_class = "%s/%s_class.png" % (OUTPUT_DIR, os.path.basename(fname)[0:-4])

#     print(f"working on file: \t {fname}")

    # if not args.force and os.path.exists(newfname_class):
    #     print("Skipping as output file exists")
    #     continue
    #
    # cv2.imwrite(newfname_class, np.zeros(shape=(1, 1)))

    
#     io = cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
#     io = cv2.resize(io, (0, 0), fx=1, fy=1)

#     io_shape_orig = np.array(io.shape)
    
#     #add half the stride as padding around the image, so that we can crop it away later
#     io = np.pad(io, [(stride_size//2, stride_size//2), (stride_size//2, stride_size//2), (0, 0)], mode="reflect")
    
#     io_shape_wpad = np.array(io.shape)
    
#     #pad to match an exact multiple of unet patch size, otherwise last row/column are lost
#     npad0 = int(np.ceil(io_shape_wpad[0] / patch_size) * patch_size - io_shape_wpad[0])
#     npad1 = int(np.ceil(io_shape_wpad[1] / patch_size) * patch_size - io_shape_wpad[1])

#     io = np.pad(io, [(0, npad0), (0, npad1), (0, 0)], mode="constant")

#     arr_out = sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)
    DL = torch.tensor(testTable.root.DLfeats[i,:].reshape(1,300)).to(device)
    HC = torch.tensor(testTable.root.HCFeats[i,:].reshape(1,10)).to(device)
#     arr_out = sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size),stride_size)
#     arr_out_shape = arr_out.shape
#     arr_out = arr_out.reshape(-1,patch_size,patch_size,1)

    #in case we have a large network, lets cut the list of tiles into batches
    output = np.zeros((0,checkpoint["num_classes"]))
    output_batch = model(DL,HC)
    output = np.append(output,output_batch.detach().cpu().numpy(),axis=0)
    
#     for dl_arr,hc_arr in divide_batch(DL,HC,batch_size):
        
        
#         hc = torch.from_numpy(hc_arr).type('torch.FloatTensor').to(device)

#         # ---- get results
        

#         # --- pull from GPU and append to rest of output 
#         output_batch = output_batch.detach().cpu().numpy()
        


#     print(output.shape)
    saveScore.append(output)
#     tileclass = np.argmax(output, axis=1)
#     print(output)
#     print(tileclass)
#     asdab
#     predc,predccounts=np.unique(tileclass, return_counts=True)
#     print(f'predcounts{predccounts}')
# #     if len(predccounts)==1:
# #         score = 0
# #     else:
# #         score = predccounts[1]/sum(predccounts)
# #     classPredDetailed.write(f"working on file: \t {fname}\n")
    
# #     print(f" score: \t{score}")
#     for c,cc in zip(predc,predccounts):
#         score = cc/(output.shape[0])
#         classPredDetailed.write(f"class/count: \t{c}\t{cc}\n")
#         print(f"class/count: \t{c}\t{cc}")
#         print(f"class {c} score: \t{score}")
        


#     classScore.write(f"{score}\n")
#     print(f"predicted class argmax([cc0,cc1]):\t{predc[np.argmax(predccounts)]}")
#     classPred.write(f"{predc[np.argmax(predccounts)]}\n")
# #     classPredDetailed.write("\n")
#     print(f"predicted class score>0.5:\t{(score>0.5)*1}")
#     classPredScore.write(f"{(score>0.5)*1}\n")
    
classPred.close()
# classPredDetailed.close()
classScore.close()
classPredScore.close()
ss = np.array(saveScore)
np.savez(OUTPUT_DIR+"classPred.npz", x=ss)


e:\mvp\venv\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


e:\mvp\venv\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [36]:
OUTPUT_DIR+"classPred.npz"


'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\OrganSiteClassifying\\FusionNew\\Test1Out\\classPred.npz'

In [22]:
testTable.root.DLfeats.shape

(14502, 300)

In [ ]:
np.savez("classPred032322.npz", x=ss)

In [ ]:
OUTPUT_DIR+"classPred.npz"

In [ ]:
OUTPUT_DIR+"classPred.npz"

In [ ]:
tileclass = np.argmax(output, axis=1)
predc,predccounts=np.unique(tileclass, return_counts=True)

In [ ]:
tileclass

In [ ]:
predc

In [ ]:
miss = np.setdiff1d([0, 1, 2, 3], predc)
np.where([0, 1, 2, 3]== miss)

In [ ]:
predccounts

In [ ]:
for c,cc in zip(predc,predccounts):
    score = cc/(output.shape[0])
    classPredDetailed.write(f"class/count: \t{c}\t{cc}\n")
    print(f"class/count: \t{c}\t{cc}")
    print(f"class {c} score: \t{score}")

In [ ]:
output.shape[0]

In [ ]:
cc

In [ ]:
output

In [ ]:
# ----- load network
device = torch.device(0 if 0!=-2 and torch.cuda.is_available() else 'cpu')

checkpoint = torch.load(mdlPath, map_location=lambda storage, loc: storage) #load checkpoint to CPU and then put to device https://discuss.pytorch.org/t/saving-and-loading-torch-models-on-2-machines-with-different-number-of-gpu-devices/6666

model = DenseNet(growth_rate=checkpoint["growth_rate"], block_config=checkpoint["block_config"],
                 num_init_features=checkpoint["num_init_features"], bn_size=checkpoint["bn_size"],
                 drop_rate=checkpoint["drop_rate"], num_classes=checkpoint["num_classes"]).to(device)

model.load_state_dict(checkpoint["model_dict"])
model.eval()

print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

# ----- get file list

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

files = []
# basepath = args.basepath  #
# basepath = basepath + os.sep if len(
#     basepath) > 0 else ""  # if the user supplied a different basepath, make sure it ends with an os.sep

# if len(args.input_pattern) > 1:  # bash has sent us a list of files
#     files = args.input_pattern
# elif args.input_pattern[0].endswith("tsv"):  # user sent us an input file
#     # load first column here and store into files
#     print('loading tsv file...')
#     with open(args.input_pattern[0], 'r') as f: #added encoding="ISO-8859-1" to fix UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte
#         for line in f:
#             print(line)
#             if line[0] == "#":
#                 continue
#                 print('Continue')
#             files.append(basepath + line.strip().split("\t")[0])
# else:  # user sent us a wildcard, need to use glob to find files
#     files = glob.glob(args.basepath + args.input_pattern[0])
files = glob.glob(base_path + input_pattern)

In [ ]:
files

In [ ]:
# base_path + input_pattern
# files = [ 'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_12033.png','E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13569.png',
#  'E:\\MvP\\FinalExperimentWithPancreas\\NewRNG\\Run3\\DeepLearningHeatmap\\Val\\Pri\\40P (2)_100097_13825.png']

In [ ]:
# +
# ------ work on files
#print(files)
classPred= open(OUTPUT_DIR+"\classPred.txt","w+")
classPredScore= open(OUTPUT_DIR+"\classPredScore.txt","w+")
classScore = open(OUTPUT_DIR+"\classScore.txt","w+")
# classPredDetailed = open(OUTPUT_DIR+"\classPredDetailed.txt","w+")
for fname in files:

    fname = fname.strip()
    newfname_class = "%s/%s_class.png" % (OUTPUT_DIR, os.path.basename(fname)[0:-4])

    print(f"working on file: \t {fname}")

    # if not args.force and os.path.exists(newfname_class):
    #     print("Skipping as output file exists")
    #     continue
    #
    # cv2.imwrite(newfname_class, np.zeros(shape=(1, 1)))

    
    io = cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
    io = cv2.resize(io, (0, 0), fx=1, fy=1)

    io_shape_orig = np.array(io.shape)
    
    #add half the stride as padding around the image, so that we can crop it away later
    io = np.pad(io, [(stride_size//2, stride_size//2), (stride_size//2, stride_size//2), (0, 0)], mode="reflect")
    
    io_shape_wpad = np.array(io.shape)
    
    #pad to match an exact multiple of unet patch size, otherwise last row/column are lost
    npad0 = int(np.ceil(io_shape_wpad[0] / patch_size) * patch_size - io_shape_wpad[0])
    npad1 = int(np.ceil(io_shape_wpad[1] / patch_size) * patch_size - io_shape_wpad[1])

    io = np.pad(io, [(0, npad0), (0, npad1), (0, 0)], mode="constant")

    arr_out = sklearn.feature_extraction.image.extract_patches(io,(patch_size,patch_size,3),stride_size)
    arr_out_shape = arr_out.shape
    arr_out = arr_out.reshape(-1,patch_size,patch_size,3)

    #in case we have a large network, lets cut the list of tiles into batches
    output = np.zeros((0,checkpoint["num_classes"]))
    
    for batch_arr in divide_batch(arr_out,batch_size):
        
        arr_out_gpu = torch.from_numpy(batch_arr.transpose(0, 3, 1, 2) / 255).type('torch.FloatTensor').to(device)

        # ---- get results
        output_batch = model(arr_out_gpu)

        # --- pull from GPU and append to rest of output 
        output_batch = output_batch.detach().cpu().numpy()
        output = np.append(output,output_batch,axis=0)


    print(output.shape)
    tileclass = np.argmax(output, axis=1)
    predc,predccounts=np.unique(tileclass, return_counts=True)
    print(f'predcounts{predccounts}')
    if len(predccounts)==1:
        score = 0
    else:
        score = predccounts[1]/sum(predccounts)
#     classPredDetailed.write(f"working on file: \t {fname}\n")
    
    print(f" score: \t{score}")
#     for c,cc in zip(predc,predccounts):
#         score = cc/(output.shape[0])
#         classPredDetailed.write(f"class/count: \t{c}\t{cc}\n")
#         print(f"class/count: \t{c}\t{cc}")
#         print(f"class {c} score: \t{score}")
        


    classScore.write(f"{score}\n")
    print(f"predicted class argmax([cc0,cc1]):\t{predc[np.argmax(predccounts)]}")
    classPred.write(f"{predc[np.argmax(predccounts)]}\n")
#     classPredDetailed.write("\n")
    print(f"predicted class score>0.5:\t{(score>0.5)*1}")
    classPredScore.write(f"{(score>0.5)*1}\n")
    
classPred.close()
# classPredDetailed.close()
classScore.close()
classPredScore.close()

In [ ]:
sum([6, 3])